In [2]:
from pathlib import Path

import pandas as pd

from formula import parse

from antlr4.error.Errors import ParseCancellationException

import json

In [3]:
v = {}
tm = {}

In [10]:
with open('phosphors.csv','w') as f:
    
    df = pd.read_csv('csv/300.clean.csv')
    df['formula'] = df['formula'].apply(lambda x: x + ':Eu2+')
    df.to_csv(f, index=None)

    for csv in Path('csv').glob('10*.csv'):
        df = pd.read_csv(csv, header=None)
        df.to_csv(f, index=None, header=None)

In [16]:
df = pd.read_csv('phosphors.csv')
df['lambda'] = pd.to_numeric(df['lambda'], errors='coerce')
df = df.dropna()

js = df.to_dict(orient='records')
js2 = []
for j in js:
    try:
        formula = j['formula']
        # formula should be in the form of 'base:dopant'
        assert ':' in formula
        # dopant should be Eu2+
        dopant = formula.split(':')[1]
        assert 'Eu2' in dopant
        # base should be a valid formula
        base = formula.split(':')[0]
        stoichiometry = parse(base)
        # all stoichiometry numbers should be close to ints, and are converted so
        for ele, num in stoichiometry.items():
            assert abs(num - round(num)) < 0.25
            stoichiometry[ele] = round(num)
        j['stoichiometry'] = stoichiometry
        # lambda should be float
        j['lambda'] = float(j['lambda'])
        js2.append(j)
    except (AssertionError, ParseCancellationException) as e:
        print('[' + formula + '] ' + repr(e))

[BaMg1.5Al10.5O18.25:Eu2+] AssertionError()
[SrMgAl11O18.5:Eu2+] AssertionError()
[La0.86Al11.9O19.4:Eu2+] AssertionError()
[Gd1−xSr2AlO5:xCe3+] AssertionError()
[CMAS:Ce3+,Dy3+] AssertionError()
[CMAS:Dy3+] AssertionError()
[CMAS:Dy3+] AssertionError()
[CMAS:Dy3+] AssertionError()
[CMAS: 1 m% Ce3+, xDy3+ (x=0.1, 0.3, 0.5 and 1 mol%)] AssertionError()
[CMAS: 1 m% Ce3+, 0.5 m% Dy3+] AssertionError()
[CMAS: 1 m% Ce3+, 0.5 m% Dy3+] AssertionError()
[CMAS: 1 m% Ce3+, 0.5 m% Dy3+] AssertionError()
[Ca20Mg3Al26Si3O68: 1 m% Ce3+] AssertionError()
[Ca20Mg3Al26Si3O68: 0.5 m% Dy3+] AssertionError()
[Ca20Mg3Al26Si3O68: 0.5 m% Dy3+] AssertionError()
[Ca20Mg3Al26Si3O68: 0.5 m% Dy3+] AssertionError()
[Ca20Mg3Al26Si3O68: 1 m% Ce3+ 0.5 m% Dy3+] AssertionError()
[Ca20Mg3Al26Si3O68: 1 m% Ce3+ 0.5 m% Dy3+] AssertionError()
[Ca20Mg3Al26Si3O68: 1 m% Ce3+ 0.5 m% Dy3+] AssertionError()
[Ba2.96Eu0.04P4O13] AssertionError()
[Ba2.96Eu0.04P4O13] AssertionError()
[Lu3MgAl3SiO12 : Ce3+] AssertionError()
[Lu2Y1MgAl

In [17]:
with open('phosphors.json', 'w') as f:
    json.dump(js2, f)